In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import docx
from string import punctuation
import re
import csv 
import os
from simhash import Simhash, SimhashIndex

In [6]:
from src.utils import search_API, fetch_details, create_csv_path, collectFromEndnote, collectFromTables

In [12]:
rba_path = os.path.abspath('data/1. Risicoanalyse kinderformularium gentamicine nieuw.docx')

In [13]:
csv_path = create_csv_path(rba_path)

In [16]:
csv_path

'/Users/sekiz/workspaces/paper-kinderformularium/data/csv/1. Risicoanalyse kinderformularium gentamicine nieuw.csv'

In [14]:
referenceList = collectFromEndnote(rba_path)

In [15]:
len(referenceList)

26

In [11]:
def writeFromEndnoteTOCsv(referenceList, csv_path):
    header = ['record_id', 'title', 'abstract', 'doi', 'final_included']
    with open(csv_path, 'w', newline='') as csvfile:
        cw = csv.writer(csvfile, delimiter=',')
        cw.writerow(header)

    #Loop over the reference list in the docx file
    for ref in referenceList:
        search_query = ref['p_doi'] if ref['p_doi'] else ref['p_title'] 
    #     print('#search_query', search_query)
        try:
            PubmedSearchResults = search_API(search_query)
            id_list = PubmedSearchResults['IdList']
            #todo If id_list is empty find a ways to reproduce it
            papers = fetch_details(id_list)
            referenceFound = False

            for paperIndex, paper in enumerate(papers['PubmedArticle']):
                paperTitle = paper['MedlineCitation']['Article']['ArticleTitle'].strip(punctuation).strip()
                if (Simhash(paperTitle).distance(Simhash(ref['p_title'])) <= 5):
                    # Todo Add Author confirmation as well
                    doi = ''
                    for idn, ids in enumerate(papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList']):
                        if papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn].attributes['IdType'] == 'doi':
                            doi = papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn]

                    abstract = ''
                    if 'Abstract' in paper['MedlineCitation']['Article']:
                        abstract = ' '.join([str(x) for x in paper['MedlineCitation']['Article']['Abstract']['AbstractText']])

                    data = [id_list[paperIndex], paperTitle, abstract, doi, 1]
                    with open(csv_path, 'a') as csvfile:
                        cw = csv.writer(csvfile, delimiter=',')
                        cw.writerow(data)
                    referenceFound = True 
                    break
            if not referenceFound:
                if (len(search_query.split(' ')) < 8):
                    search_query = search_query + ' ' + ref['p_authors'].split(' ')[0]
                    print('!!!!!!!!!!!!!!!', search_query)
    #                 elif (len(search_query.split(' ')) >= 8):
    #                 If it is longer divide it half first then remove 1 by 1


        except:
            print('!!!! The reference could not be found automaticaly: ', ref['p_title'])


In [24]:
writeFromEndnoteTOCsv(referenceList, csv_path)

!!!! The reference could not be found automaticaly:  NVK werkboek Infectieziekten bij kinderen
!!!! The reference could not be found automaticaly:  TDM gentamicine 8-7-2015
!!!! The reference could not be found automaticaly:  Werkboek kindernefrologie 2010
!!!!!!!!!!!!!!! Drug prescribing in renal failure Aronoff
!!!! The reference could not be found automaticaly:  Population pharmacokinetic study of gentamicin in a large cohort of premature and term neonates


### TODO   
Make a function that creates new search term for the articles that couldnt be found and make kind of recursive to send the new search term to API every time until it is found  
- Preferably 3 trial would be enough   
- If the length is shorter than 5 or 6 just remove one word
- if it is too long first divide it to half then remove one word
- if it is too short try sending with a author name


In [20]:
search_query = 'Population pharmacokinetic study of gentamicin '

In [21]:
PubmedSearchResults = search_API(search_query)
id_list = PubmedSearchResults['IdList']
print(len(id_list), id_list)

20 ['31102287', '29957666', '29521784', '28062683', '29704547', '28533244', '28038961', '31286535', '30300445', '34362436', '24522409', '34625981', '24938850', '30917981', '28475651', '25225917', '30980613', '30882729', '27420805', '32394828']


In [22]:
papers = fetch_details(id_list)

In [113]:
trial = 1
if not id_list:
    new_search_query = createNewSearchTerm(ref, trial)
    trial += 1

In [131]:
new_search_query = createNewSearchTerm(ref, 7)

In [132]:
new_search_query

False

In [130]:
def createNewSearchTerm(searchTerm, trial):
    if len(searchTerm.split(' '))/2 > 2 and  trial == 1:
        newTerm = ' '.join(str(x) for x in searchTerm.split(' ')[0:int(len(searchTerm.split(' '))/2)])
        return newTerm
    elif (len(searchTerm.split(' '))/2-trial +1) > 2 and trial != 1:
        newTerm = ' '.join(str(x) for x in searchTerm.split(' ')[0:int(len(searchTerm.split(' '))/2)-trial +1])
        return newTerm
    else:
        return False

In [25]:
papers = fetch_details(id_list)

In [ ]:
### version also uses runs not only paragraphs
    
doc = Document(rba_path)
table_headers = ['bron', 'bewijs', 'effect', 'opmerkingen', 'source', 'evidence', 'effect', 'remarks']
data = []
referenceListfromTable = []
authorListfromTable = []
for table in doc.tables:
    for i, row in enumerate(table.rows):

        text = [cell.text.lower() for cell in row.cells]

        if i == 0:
            effect_index = text.index('effect') if 'effect' in text else None
            continue

#         if effect_index is None:
#             continue

        for j, cell in enumerate(row.cells):

            if j==effect_index:

                bold_text = ''
                authorsText = ''
                authorsTextRaw = ''
                untilSummary = False
                
                for paragraph in cell.paragraphs:
                    for k, run in enumerate(paragraph.runs):
                        if run.bold and authorsTextRaw == '':
                            justReadBold = True
                            bold_text += run.text.strip() + ' '
                        elif ('et al' in run.text and not paragraph.runs[k-1].bold 
                                  and authorsTextRaw == ''):

                            if (paragraph.runs[k-1].text not in run.text):
                                authorsTextRaw = paragraph.runs[k-1].text + run.text
                            else:
                                authorsTextRaw = run.text
                            authorsText = authorsTextRaw.strip(punctuation).strip()
                            authorsText = re.sub(r'et al.*\d*', '',authorsText).strip()
                        
                        
                        elif run.underline and (run.text.lower().startswith('samenvatting') 
                          or run.text.lower().startswith('summary')):
                            print(run.text)
#                             print(authorsText, authorsTextRaw)
#                         else:
#                             print(run.text)
                if bold_text.strip() != ''  and re.search(r'^[0-9].[0-9]', bold_text) is None: 
                    referenceListfromTable.append(bold_text.strip().replace('\xa0',' '))
                    authorListfromTable.append({
                        'title': bold_text.strip().replace('\xa0',' '),
#                         'authors': authorsText,
                        'authors': authorsTextRaw
                    })

In [33]:
referencesFromTables = collectFromTables(rba_path)

In [34]:
referencesFromTables

[{'p_title': 'Gentamicin and tobramycin in neonates: compari-son of a new extended dosing interval regimen with a traditional multiple daily dosing regimen',
  'p_authors': ' Avent ML et al.'},
 {'p_title': 'Use of 4-mg/kg/24-hour empiric aminoglycoside dosing in preoperative neonates with congenital heart disease',
  'p_authors': ' Moffett BS et al.'},
 {'p_title': 'Impact of small-for-gestational age (SGA) status on gentamicin pharmacokinetics in neonates',
  'p_authors': ' Lulic‐Botica M et al.'},
 {'p_title': 'Pharmacokinetics of gentamicin in newborns with moderate-to-severe hypoxic-ischemic encephalopathy undergoing therapeutic hypothermia',
  'p_authors': ' Ting JY et al.'},
 {'p_title': 'Standard or individualised gentamicin dosing for neonates',
  'p_authors': ' Caldwell N et al.'},
 {'p_title': 'Novel model-based dosing guidelines for gentamicin and tobramycin in preterm and term neonates',
  'p_authors': ' Valitalo PA et al.'},
 {'p_title': 'Gentamicin trough levels using a 

In [ ]:
def writeFromTablesToCsv():
    header = ['record_id', 'title', 'abstract', 'doi', 'final_included']
    with open(csv_path, 'w', newline='') as csvfile:
        cw = csv.writer(csvfile, delimiter=',')
        cw.writerow(header)

    #Loop over the reference list in the docx file
    for ref in referencesFromTables:
        ref = ref['title'].strip()

        if (ref.lower().startswith('reference') or ref.lower().startswith('referen')):
            print('There might be some problem with the reference below. \
                Please check and add the reference if it is not on the list',ref)
            continue
        elif (ref.strip() == ''):
            print('There might be some problem with the reference below. \
                Please check and add the reference if it is not on the list',ref)
            continue
        else:
            try:
                papers = searchArticle(ref)
                referenceFound = False

                for paperIndex, paper in enumerate(papers['PubmedArticle']):
                    paperTitle = paper['MedlineCitation']['Article']['ArticleTitle'].strip(punctuation).strip()
                    if (paperTitle == textTitle.strip(punctuation).strip()):
                        doi = ''
                        for idn, ids in enumerate(papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList']):
                            if papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn].attributes['IdType'] == 'doi':
                                doi = papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn]

                        abstract = ''
                        if 'Abstract' in paper['MedlineCitation']['Article']:
                            abstract = ' '.join([str(x) for x in paper['MedlineCitation']['Article']['Abstract']['AbstractText']])

                        data = [id_list[paperIndex], paperTitle, abstract, doi, 1]
                        with open(csv_path, 'a') as csvfile:
                            cw = csv.writer(csvfile, delimiter=',')
                            cw.writerow(data)
                        referenceFound = True 
                        break

            except:
                print('!!!! The reference could not be found automaticaly, please add it to the list manually')
                print(ref)